In [1]:
import os
from pathlib import Path

import pandas as pd

from utils import get_anndata

In [2]:
data_dir = Path("./data/LIBD_DLPFC")
result_dir = Path("./results/SpiceMix")

seed = 42

In [3]:
result_dir.mkdir(parents=True, exist_ok=True)

In [4]:
metadata = pd.read_table(
    data_dir / "samples.tsv", usecols=["directory", "n_clusters"]
).set_index("directory")

In [5]:
n_genes = 3_000
n_pcs = 30
k = 20

In [6]:
sample = metadata.loc["Br8100_151673", :]

In [7]:
# generate the input file and preprocess
import scanpy as sc
from popari.components import PopariDataset
from popari.io import save_anndata

adata = get_anndata(data_dir / sample.name)

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=n_genes)

adata = PopariDataset(adata[:, adata.var["highly_variable"]], "processed")
adata.compute_spatial_neighbors()

save_anndata(result_dir / f"{sample.name}.h5ad", [adata])

/dh-projects/ag-ishaque/analysis/muellni/envs/spicemix/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/dh-projects/ag-ishaque/analysis/muellni/envs/spicemix/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/dh-projects/ag-ishaque/analysis/muellni/envs/spicemix/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


AnnData object with n_obs × n_vars = 3639 × 3000
    obs: 'row', 'col', 'batch'
    var: 'gene_version', 'gene_name', 'source', 'gene_biotype', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'spatial_neighbors', 'adjacency_matrix', 'dataset_name'
    obsm: 'spatial'

In [7]:
conda_path = "~/miniconda3/bin/activate"

log_dir = result_dir / "logs"
log_dir.mkdir(parents=True, exist_ok=True)


path = Path(os.getcwd())

In [9]:
sample_log_dir = log_dir.resolve() / sample.name
out_dir = result_dir / sample.name

sample_log_dir.mkdir(parents=True, exist_ok=True)
out_dir.mkdir(parents=True, exist_ok=True)

In [10]:
conda_env = "spicemix"
conda_cmd = f"source {conda_path} {conda_env}"

for reg in [12, 10, 8, 6, 4]:
    cmd = (
        f"{path/'spicemix.py'} "
        f"-i {result_dir / f'{sample.name}.h5ad'} "
        f"-o {out_dir/f'spicemix_exp{reg}.h5ad'} "
        f"--reg_exp {reg} "
        f"--k {k} "
        f"--seed {seed} "
    )

    os.popen(
        f"sbatch -J {sample.name} --mem=64G -n 8 -N 1 --gres=gpu:1 --partition graphical-legacy --time=06:00:00 "
        f"-o {sample_log_dir/f'spicemix_{reg}'}.txt "
        f'--wrap="{conda_cmd} && {cmd}" '
    ).read()